<a href="https://colab.research.google.com/github/Gh5al/CDMO/blob/main/MIP/MIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install highspy
!apt-get install coinor-cbc
!apt-get install -y -qq glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 ht

3d decision variable + MTZ constraint to prevent subtours<br>
3 solvers are used:
- GLPK
- CBC
- HiGHS

In [46]:
from pyomo.environ import *
from pyomo.contrib.appsi.solvers import Cbc, Highs
import time

In [41]:
filename = "./inst06.dat"
dist = []
with open(filename,'r') as f:
  m = int(f.readline().strip())
  n = int(f.readline().strip())
  capacity = [int(s) for s in f.readline().strip().split()]
  size = [int(s) for s in f.readline().strip().split()]
  for i in range(n+1):
    dist.append([int(s) for s in f.readline().strip().split()])

print(m,n)
print(capacity)
print(size)
print(dist)

6 8
[200, 100, 200, 300, 200, 100]
[16, 25, 21, 14, 19, 14, 6, 20]
[[0, 80, 131, 22, 41, 127, 87, 48, 113], [60, 0, 85, 82, 101, 81, 53, 106, 57], [141, 83, 0, 129, 182, 4, 64, 189, 28], [22, 82, 129, 0, 55, 125, 65, 66, 101], [41, 99, 172, 55, 0, 168, 118, 11, 154], [137, 81, 4, 125, 178, 0, 60, 185, 32], [77, 63, 64, 65, 118, 60, 0, 125, 36], [48, 126, 179, 60, 58, 175, 125, 0, 161], [113, 57, 28, 101, 154, 24, 36, 161, 0]]


In [42]:
#3d decision variable xkij, if the courier k goes from i to j = 1 otherwise 0 and MTZ constraint to prevent subtours
model = ConcreteModel()
model.K = Set(initialize=range(m)) #couriers
model.I = Set(initialize=range(n)) #items
model.loc=Set(initialize=range(n+1)) #locations

#variables
model.x = Var(model.K, model.loc,model.loc,domain=Binary) #decision variable
model.o = Var(model.K, model.I,domain=PositiveIntegers)  # order variable to prevent subtour
model.dist = Var(model.K, domain=PositiveIntegers) #memorize the distance for each courier
model.obj = Var(domain=PositiveIntegers)  # the objective function = max distance to be minimized

#constraints
#each item should be delivered by only one courier
def one_item_delivery_rule(model,j):
    return sum(model.x[k,i,j] for k in model.K for i in model.loc if i!=j) == 1
model.one_item_delivery = Constraint(model.I, rule=one_item_delivery_rule)

#enter and exit from node, number of times exit a node i = number of times enter a node i
def enter_exit_rule(model,k,i):
    return sum(model.x[k,i,j] for j in model.loc) == sum(model.x[k,j,i] for j in model.loc)
model.enter_exit = Constraint(model.K, model.loc, rule=enter_exit_rule)

#each courier start from depot exactly once
def leave_depot_rule(model,k):
    return sum(model.x[k,n,j] for j in model.I) == 1
model.depot_leave = Constraint(model.K, rule=leave_depot_rule)

#each courier return to depot exactly once
def return_depot_rule(model,k):
    return sum(model.x[k,i,n] for i in model.I) == 1
model.depot_return = Constraint(model.K, rule=return_depot_rule)

#avoid self-loop
def self_loop_rule(model,k,i):
    return model.x[k,i,i] == 0
model.self_loop = Constraint(model.K, model.loc, rule=self_loop_rule)


#capacity constraint
def capacity_rule(model,k):
    return sum(size[i]*(model.x[k,i,j]) for i in model.I for j in model.loc if i!=j) <= capacity[k]
model.capacity = Constraint(model.K, rule=capacity_rule)

#prevent subtours with MTZ constraint, used in TSP problem
#https://phabe.ch/2021/09/19/tsp-subtour-elimination-by-miller-tucker-zemlin-constraint/
model.order = ConstraintList()
for k in model.K:
    for i in model.I:
      for j in model.I:
        if i != j:
          #use Big M notation(M=2*n)
          model.order.add(model.o[k, j] - model.o[k,i] >= 1 -(1-model.x[k,i,j])*2*n)

#compute distance
def total_distance_rule(model,k):
    return model.dist[k] == sum(dist[i][j] * model.x[k, i, j] for i in model.loc for j in model.loc if i != j)
model.total_distance_constraint = Constraint(model.K, rule=total_distance_rule)

#take the max distance travelled by any courier as obj
def max_distance_constraint_rule(model, k):
    return model.obj >= model.dist[k]
model.max_distance_constraint = Constraint(model.K, rule=max_distance_constraint_rule)

#lowerbound constraint
def lower_bound_rule(model,i):
    return model.obj >= (dist[n][i] + dist[i][n])
model.lower_bound = Constraint(model.I, rule=lower_bound_rule)
#objective
model.objective = Objective(expr = model.obj, sense=minimize)



In [43]:
#GLPK SOLVER
solver = SolverFactory('glpk')
solver.options['tmlim'] = 300
results = solver.solve(model,tee=False)
print(results)
print(model.obj.value)


Problem: 
- Name: unknown
  Lower bound: 322.0
  Upper bound: 322.0
  Number of objectives: 1
  Number of constraints: 490
  Number of variables: 541
  Number of nonzeros: 3248
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 21
      Number of created subproblems: 21
  Error rc: 0
  Time: 0.03046417236328125
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

322.0


In [47]:
#CBC SOLVER
start_time = time.time()
solver = Cbc()
solver.config.time_limit = 300
results = solver.solve(model)
final_time = time.time() - start_time
print(results)
print(final_time)
print(model.obj.value)

termination_condition: TerminationCondition.optimal
best_feasible_objective: 322.0
best_objective_bound: -inf
0.28536057472229004
322.0


In [48]:
#HIGHS SOLVER
start_time = time.time()
solver = Highs()
solver.config.time_limit = 300
results = solver.solve(model)
final_time = time.time() - start_time
print(results)
print(final_time)
print(model.obj.value)

termination_condition: TerminationCondition.optimal
best_feasible_objective: 322.0
best_objective_bound: 322.0
1.3741250038146973
322.0


In [53]:
print(model.I.data())
sol = []
for k in model.K:
    sub_sol = []
    print(f"Courier {k}:")
    for j in model.loc:
      if value(model.x[k,n,j]) > 0.5:
        first = j
        sub_sol.append(first+1)
    for i in model.loc:
        for j in model.loc:

            if value(model.x[k, i, j]) > 0.5:
                #print(value(model.x[k, i, j]))
                print(f"travels from {i} to {j}")
    #route extraction(items delivered by the courier k following the delivery order)
    succ=0
    prec=first
    while(True):
      for i in model.loc:
        if value(model.x[k,prec,i]>0.5):
          #print(f" from {prec} to {i}")
          if i == n:
            succ=n
          prec=i
          break
      if succ == n:
        break
      sub_sol.append(prec+1)
    print(sub_sol)
    sol.append(sub_sol)
print(sol)


(0, 1, 2, 3, 4, 5, 6, 7)
Courier 0:
travels from 0 to 8
travels from 7 to 0
travels from 8 to 7
[8, 1]
Courier 1:
travels from 3 to 8
travels from 4 to 3
travels from 8 to 4
[5, 4]
Courier 2:
travels from 1 to 8
travels from 8 to 1
[2]
Courier 3:
travels from 5 to 8
travels from 8 to 5
[6]
Courier 4:
travels from 6 to 8
travels from 8 to 6
[7]
Courier 5:
travels from 2 to 8
travels from 8 to 2
[3]
[[8, 1], [5, 4], [2], [6], [7], [3]]


In [54]:
import json
import os
import math

# Example values
instance_id = 3
approach = "MIP"
approach_name = "highs"

if final_time<300:
  optimal = "true"
  time = math.floor(final_time)
else:
  optimal = "false"
  time = 300

objective_value = model.obj.value
solution = sol.copy()

# Create JSON structure
res = {
    approach_name: {
        "time": time,
        "optimal": optimal,
        "obj": int(objective_value),
        "sol": sol
    }
}
# Make sure directory exists
out_dir = f"res/{approach}"
os.makedirs(out_dir,exist_ok=True)

# Save to file
with open(f"{out_dir}/{instance_id}.json", "w") as f:
    json.dump(res, f)
